In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore
import pinecone
from pinecone import Pinecone, ServerlessSpec

from uuid import uuid4
import os
import time
from dotenv import load_dotenv

/opt/homebrew/Caskroom/miniconda/base/envs/med-ast/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()

True

In [3]:
# Etract data from PDF
def load_pdf(data):
   loader =  DirectoryLoader(path=data, 
                            glob='*.pdf',
                            loader_cls=PyPDFLoader)
   documents = loader.load()

   return documents

In [4]:
extracted_data = load_pdf('data/')
extracted_data

[Document(metadata={'source': 'data/gale_encyclopedia_of_medicine_vol4.pdf', 'page': 0}, page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION'),
 Document(metadata={'source': 'data/gale_encyclopedia_of_medicine_vol4.pdf', 'page': 1}, page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nN-S4'),
 Document(metadata={'source': 'data/gale_encyclopedia_of_medicine_vol4.pdf', 'page': 2}, page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow, Manager, Imaging and Multimedia\nContent\nRobyn V . Young, Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imag

In [5]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, 
                                                chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents=extracted_data)
    
    return text_chunks

In [6]:
text_chunks = text_split(extracted_data)
len(text_chunks)
# text_chunks

10484

In [7]:
# Embedding model
def download_embeddings_model():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings



In [8]:
embeddings = download_embeddings_model()

/var/folders/l5/nfl8tw2j4jb7l_5plk_dxddr0000gn/T/ipykernel_79986/4235689176.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [9]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [10]:
query_result

[-0.03447728604078293,
 0.031023267656564713,
 0.006734997034072876,
 0.02610897831618786,
 -0.0393621064722538,
 -0.16030247509479523,
 0.066924087703228,
 -0.00644137617200613,
 -0.04745044559240341,
 0.014758842997252941,
 0.07087539881467819,
 0.05552750080823898,
 0.019193340092897415,
 -0.02625138871371746,
 -0.01010960340499878,
 -0.026940476149320602,
 0.022307490929961205,
 -0.022226663306355476,
 -0.149692565202713,
 -0.01749301701784134,
 0.007676353678107262,
 0.05435230955481529,
 0.003254495793953538,
 0.03172589838504791,
 -0.08462144434452057,
 -0.0294061116874218,
 0.05159566551446915,
 0.048124078661203384,
 -0.0033147772774100304,
 -0.058279260993003845,
 0.04196932166814804,
 0.022210663184523582,
 0.128188818693161,
 -0.022338958457112312,
 -0.011656227521598339,
 0.06292832642793655,
 -0.03287624567747116,
 -0.09122596681118011,
 -0.03117532841861248,
 0.05269956216216087,
 0.04703476279973984,
 -0.0842030718922615,
 -0.030056225135922432,
 -0.020744914188981056,


In [11]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_api_env = 'gcp-starter'

pc = Pinecone(api_key=pinecone_api_key)

In [12]:
index_name = 'medical-assistant-vector-db'

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [13]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [14]:
uuids = [str(uuid4()) for _ in range(len(text_chunks))]

vector_store.add_documents(documents=text_chunks, ids=uuids)

['f32d3869-c608-44b7-b3f8-94e063da7eb2',
 '89a99eac-66ab-4ca2-97bb-3070596b7ddd',
 '7906909e-0f0c-4b6c-8557-488458793a57',
 '44f2b330-a567-4ae7-a8d4-60414b414822',
 '7c53c5e0-2899-4ccf-9c4a-863157df3118',
 '65d086e6-2762-49a4-b3d6-444d17dbe144',
 'd39db825-5e13-4861-b28c-46ed8f1f3a31',
 '094c9e9e-0492-4230-8919-cf6332b4cbc3',
 '2d1762fd-655e-4780-b806-45f608938ae6',
 'abb34978-fdc2-46af-ab80-44e22e8171ba',
 '17742a85-d46e-40bd-9db6-44282c48b204',
 '6df17874-d89d-4023-89f4-f98b1ae91a38',
 'c1f1279e-9803-4fe5-9112-d47b79a6fb82',
 'a863b9a3-770d-41db-8d3f-1b2bd6fab1d2',
 '7896da16-1acd-49f8-bb63-f564b72407a8',
 '4903b22d-eb14-4511-928e-a902327f2852',
 'e2689994-74b8-4c57-b783-e20bf6b2ab0f',
 '2ee9d9c5-47e9-4ca8-ad47-10b24b90f39e',
 '9716c1d7-efa4-4a3a-a90d-cfae00cf8ce5',
 'e3f67e55-7764-4190-8864-8a65159e2df8',
 '00b009f4-91f8-4d78-8019-2193d5abcdb6',
 '4b0c94ac-f336-4ba3-882e-1e7109e054d1',
 'fd29ee38-b67d-450b-8a13-826d218807a2',
 '55fdaf42-ec60-49e8-ba69-9c0923355f5d',
 'cdcb9108-3524-

In [15]:
results = vector_store.similarity_search(
    query="What are Allergies",
    k=2,
)

for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* GALE ENCYCLOPEDIA OF MEDICINE 2 2591Physical allergy [{'page': 297.0, 'source': 'data/gale_encyclopedia_of_medicine_vol4.pdf'}]
* GALE ENCYCLOPEDIA OF MEDICINE 2 2591Physical allergy [{'page': 297.0, 'source': 'data/gale_encyclopedia_of_medicine_vol4.pdf'}]


In [16]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [17]:

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [18]:
llm = CTransformers(model="model/llama-2-7b-chat.Q4_0.gguf",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.8})

In [19]:

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vector_store.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:

while True:
    user_input = input(f"Input Prompt:")
    result = qa({"query": user_input})
    print("Response : ", result["result"])